In [1]:
import matplotlib
matplotlib.use('agg')

import os
import pickle
import numpy as np
import pandas as pd
import xarray as xr
import seaborn as sns
from keras.layers import Input, Convolution2D, Convolution1D, MaxPooling2D, Dense, Dropout, \
                          Flatten, concatenate, Activation, Reshape, \
                          UpSampling2D,ZeroPadding2D
from keras import layers

import keras
from pylab import plt
import tensorflow as tf
from matplotlib.pyplot import plot, figure
from dask.diagnostics import ProgressBar
ProgressBar().register()


import xarray as xr
import numpy as np
import cmocean # for perceptually uniform colormaps
import cartopy as cr # for geographic mapping
import cartopy.crs as ccrs # for map projections
import matplotlib.pyplot as plt # plotting tool
import cartopy.feature as cfeature # to add coastlines, land and ocean
from cartopy import config
import cartopy.crs as ccrs
from cartopy.feature import NaturalEarthFeature
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfea
from netCDF4 import Dataset
import matplotlib.pylab as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib as mpl
import datetime
import os
from datetime import datetime
import matplotlib.patheffects as PathEffects
import geocat.viz as gv
import cmaps
import geocat.datafiles as gdf
import geocat.viz as gv
import geocat.viz.util as gvutil
from scipy import stats
from mpl_toolkits.basemap import Basemap, cm, shiftgrid, addcyclic
from scipy import signal
from scipy import stats
import math
import iris
from datetime import datetime
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
from scipy import stats
import numpy as np
from scipy import signal
import numpy.polynomial.polynomial as poly
from netCDF4 import Dataset
from mpl_toolkits.basemap import Basemap
from eofs.standard import Eof
import pint
import pint_pandas

In [2]:
#Saved path
outdir='C:/Users/user/Research/Research Code/CNN/trainning/channelwise_Nonnormalised/train_1428/'
ifile_data ='C:/Users/user/Research/Research Code/CNN/Data/'
testsizepath='C:/Users/user/Research/Research Code/CNN/y_test/y_testsize_480_32_64_1.nc'
y_test_size= xr.open_dataarray(testsizepath)

ifile ='C:/Users/user/Research/Research Code/CNN/trainning/channelwise_Nonnormalised/train_1428/'
outdir_original_pred = 'C:/Users/user/Research/Research Code/CNN/CW_PDO/original/pred/'
outdir_original_test = 'C:/Users/user/Research/Research Code/CNN/CW_PDO/original/test/'
outdir_original_valid = 'C:/Users/user/Research/Research Code/CNN/CW_PDO/original/validation/'
outdir_smooth_pred = 'C:/Users/user/Research/Research Code/CNN/CW_PDO/smoothed/pred/'
outdir_smooth_test= 'C:/Users/user/Research/Research Code/CNN/CW_PDO/smoothed/test/'
outdir_smooth_valid= 'C:/Users/user/Research/Research Code/CNN/CW_PDO/smoothed/validation/'
outdir_correlation = 'C:/Users/user/Research/Research Code/CNN/CW_PDO/correlation/'
outdir_smooth_pred_label = 'C:/Users/user/Research/Research Code/CNN/CW_PDO/smoothed/pred_label/'
save_path = 'C:/Users/user/Research/Research Code/CNN/PDO/Heatmap/'

In [3]:
##Set up all functions
def weighted_areamean(ds):
    '''area mean weighted by cos of latitude'''
    weights = xr.ufuncs.cos(np.deg2rad(ds.lat))
    norm = np.sum(weights) * len(ds.lon)
    amean = (ds*weights).sum(('lat','lon')) / norm

    return amean


def acc_score(x,y):
    '''timestepwise anomaly correlation coefficient, averaged over time
        (simple version without seasonal climatoloty)'''
    assert(x.shape==y.shape)
    return np.mean([np.corrcoef(x[i].flatten(),y[i].flatten())[0,1] for i in range(len(x))])


def corr_over_time(x,y):
    '''timestepwise anomaly correlation coefficient, averaged over time
        (simple version without seasonal climatoloty)'''    
    mx = x[0:i].mean()
    my = y[0:i].mean()
    xm, ym = x[0:i]-mx, y[0:i]-my
    r_num = (xm*ym).mean()
    r_den = xm.std() * ym.std()
    r = r_num / r_den
        
    return r

def mean(x):
    avg = sum(x)/len(x)
    return round(avg,2)


def adjust(ds):
    '''transform the data'''
    z1 = np.array(ds)
    df = pd.DataFrame (z1)
    pred = df.iloc[:,0]
    return pred 

def correlation(x,y,z):
    corr_test = np.corrcoef(x [0:i],y [0:i])[0,1]
    z.append(corr_test)
    
def lowpass_filter (x,y):
    '''fraction of nyquist frequency, here  it is 10 years'''
    fs=1/12/30/24/3600 

    nyquist = fs / 2 # 0.5 times the sampling frequency
    cutoff=x # fraction of nyquist frequency, here  it is 10 years
    b, a = signal.butter(5, cutoff, btype='lowpass') #low pass filter


    dUfilt = signal.filtfilt(b, a, y)
    dUfilt=np.array(dUfilt)
    dUfilt=dUfilt.transpose()
    return dUfilt

def avg(myArray, N=12):
    '''average every 12 months'''
    cum = np.cumsum(myArray,0)
    result = cum[N-1::N]/float(N)
    result[1:] = result[1:] - result[:-1]

    remainder = myArray.shape[0] % N
    if remainder != 0:
        if remainder < myArray.shape[0]:
            lastAvg = (cum[-1]-cum[-1-remainder])/float(remainder)
        else:
            lastAvg = cum[-1]/float(remainder)
        result = np.vstack([result, lastAvg])

    return result

def detrend_dim(da,trend, dim, deg=1):
    '''detrend along a single dimension'''
    p = da.polyfit(dim=dim, deg=deg)
    fit = xr.polyval(trend[dim], p.polyfit_coefficients)
    return da - fit

In [3]:
os.system('mkdir -p '+outdir)
N_gpu = 0
print('open inputdata')

open inputdata


In [14]:
#Imported data
ds = xr.open_dataset(ifile_data+'nonnorm_Datapreprcess_ersst_1854_2022.nc')
ds

<xarray.Dataset>
Dimensions:  (time: 2028, lat: 32, lon: 64, lev: 1)
Coordinates:
  * time     (time) datetime64[ns] 1854-01-01 1854-02-01 ... 2022-12-01
  * lat      (lat) float32 64.0 62.0 60.0 58.0 56.0 ... 10.0 8.0 6.0 4.0 2.0
  * lon      (lon) float32 122.0 124.0 126.0 128.0 ... 242.0 244.0 246.0 248.0
  * lev      (lev) float64 0.0
Data variables:
    sst      (time, lat, lon, lev) float32 ...

In [15]:
##Prepared data
ds=ds['sst'].data 
ds[ds<0]=0 #treat nan values as 0
np.nan_to_num(ds,copy=False)
ds=ds/(np.nanmax(ds)) 
dates=pd.date_range(start='1854-01-01',periods=len(ds))
label=np.array(dates.month)
label=label-1

In [16]:
##Data separation
N_total = 2028
N_train = 1428
N_gab_1 = 120
N_test = 480
N_skip = N_total-N_train-N_gab_1-N_test

x = ds

In [17]:
##Verify that we have sufficient information for the requirements.
if N_skip+N_train+N_gab_1+N_test > x.shape[0]:
    raise Exception('not enough timesteps in input file!')

x = x.astype('float32')
x = x[:N_skip+N_train+N_gab_1+N_test]

lat,lon,lev=x.shape[1:4]

In [18]:
def prepare_data(lead_time):
    ''' Shift the data based on the lead time and 
    divide it into a predictor set and a predictant set. 
    Then divide it into a training set, a development set, and a test set.'''
    if lead_time == 0:
        X = x
        y = X[:]

    else:

        X = x[:]
        y = x[:]
    
  
    X_train = X[N_skip:N_skip+N_train-lead_time]
    y_train = y[N_skip:N_skip+N_train-lead_time]
    
    X_test = X[N_skip+N_train+N_gab_1:N_skip+N_train+N_gab_1+N_test]
    y_test = y[N_skip+N_train+N_gab_1:N_skip+N_train+N_gab_1+N_test]
    
       
    return X_train,y_train, X_test, y_test

In [19]:
# fixed (not-tuned params)
batch_size = 32
num_epochs = 10
pool_size = 1
drop_prob=0
conv_activation='relu'

In [20]:
def acc_score(x,y):
    '''timestepwise anomaly correlation coefficient, averaged over time
        (simple version without seasonal climatoloty)'''
    assert(x.shape==y.shape)
    return np.mean([np.corrcoef(x[i].flatten(),y[i].flatten())[0,1] for i in range(len(x))])

In [21]:
def build_model(conv_depth, kernel_size, hidden_size, n_hidden_layers, lr):

    model = keras.Sequential([
            
            ## Convolution which involves dimensionality reduction (similar to Encoder in an autoencoder)
            Convolution2D(conv_depth, kernel_size, padding='same', activation=conv_activation, input_shape=(lat,lon,lev)),
            layers.MaxPooling2D(pool_size=pool_size),
            Dropout(drop_prob),
            Convolution2D(conv_depth, kernel_size, padding='same', activation=conv_activation),
            layers.MaxPooling2D(pool_size=pool_size),
            Convolution2D(conv_depth, kernel_size, padding='same', activation=conv_activation),
            layers.MaxPooling2D(pool_size= 1),
        
        
            # end "encoder"
            
            
            # dense layers (Automatic flattening and reshaping occurs.)
            ] + [layers.Dense(hidden_size, activation='sigmoid') for i in range(n_hidden_layers)] +
             
            [
            
            
            # start "Decoder" (upsampling of the encoder above)
            Convolution2D(conv_depth, kernel_size, padding='same', activation=conv_activation),
            layers.UpSampling2D(size=pool_size),
            Convolution2D(conv_depth, kernel_size, padding='same', activation=conv_activation),
            layers.UpSampling2D(size=pool_size),
            layers.Convolution2D(lev, kernel_size, padding='same', activation=None)
            ]
            )
    
    
    optimizer= keras.optimizers.Adam(lr=lr)

    if N_gpu > 1:
        with tf.device("/cpu:0"):
            # convert the model to a model that can be trained with N_GPU GPUs
             model = keras.utils.multi_gpu_model(model, gpus=N_gpu)
             
    model.compile(loss='mean_squared_error', optimizer = optimizer)
    
    return model

# Start second training 

In [26]:
for lead_time in range(0,361):
    X_train,y_train, X_test, y_test = prepare_data(lead_time)
    
    ## The tuning process produced the best hyperparameters below.
    params = {'conv_depth': 32, 'hidden_size': 500,
              'kernel_size': 4, 'lr': 3e-05, 'n_hidden_layers': 0}

    
    print(params)
    param_string = '_'.join([str(e) for e in (N_train,num_epochs,lead_time)])
    
    
    #run the model
    model = build_model(**params)
    
    print(model.summary())
    
    
    print('start training')
    hist = model.fit(X_train, y_train,
                       batch_size = batch_size,
             verbose=1, 
             epochs = num_epochs,
             validation_data=(X_test,y_test),
             callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                        min_delta=0,
                                        patience=5, #To ensure a great deal of patience before ending
                                        verbose=0, mode='auto'),
                       keras.callbacks.ModelCheckpoint('best_weights.h5', monitor='val_loss', 
                                                    verbose=1, save_best_only=True, 
                                                    save_weights_only=True, mode='auto', period=1)]
             )
    
    print('finished training')
    
    # To ensure the best model performance from the training based on learning curve,
    # Because the early stopping callback saves the model's "patience" epochs after the best one, this is required.
    model.load_weights('best_weights.h5')
    
    # delete the file generated by Model Checkppoint
    os.system('rm best_weights.h5')
    
    
    model.save_weights(outdir+'weights_tunedparams_leadtime'+str(lead_time)+'params_'+param_string+'.h5')
  
    # reformat history
        
    hist =  hist.history

    y_test_predicted = model.predict(X_test)

    # compute accuracy
    res = []
    rmse = np.sqrt(np.mean((y_test_predicted - y_test)**2))
    acc = acc_score(y_test_predicted, y_test)

    res.append(dict(hist=hist,params=params, scores=[rmse,acc]))

    pickle.dump(res,open(outdir+'training_result_sequential'+param_string+'.pkl','wb'))
    
    # plot loss value
    plt.figure()
    plt.title('a) Channel-wise-based CNN model', fontsize=14, x=0.0, y=1.05)
    plt.xlabel('Epochs',fontsize=12)
    plt.ylabel('Loss',fontsize=12)
    plt.plot(hist['val_loss'], label='validation loss')
    plt.plot(hist['loss'], label='train loss')
    
    plt.legend()
    
    #save loss to .png
    plt.savefig(outdir+'cwcnn_history_tunedparams_leadtime'+str(lead_time)+'.png')

    pd.DataFrame(hist).to_csv(outdir+'history_tunedparams_leadtime'+str(lead_time)+'.csv')

    
    '''y_test is a xarray dataarray, but y_test_predicted is now a numpy array.
    Therefore, we convert it to an xarray with exactly the same coordinates and dims by using dim from y_test.'''
    # save the validation
    y_test_predicted_new = xr.DataArray(data=y_test_predicted, coords=y_test_size.coords, dims=y_test_size.dims)
    y_test_new = xr.DataArray(data=y_test, coords=y_test_size.coords, dims=y_test_size.dims)
    
    # save the predictions
    y_test_predicted_new.to_netcdf(outdir+'/predictions_tuned_leadtime'+str(lead_time)+'params_'+param_string+'.nc')
    y_test_new.to_netcdf(outdir+'/truevalues_tuned_leadtime'+str(lead_time)+'params_'+param_string+'.nc')
    

{'conv_depth': 32, 'hidden_size': 500, 'kernel_size': 4, 'lr': 3e-05, 'n_hidden_layers': 0}
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 32, 64, 32)        544       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 32, 64, 32)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 32, 64, 32)        0         
                                                                 
 conv2d_7 (Conv2D)           (None, 32, 64, 32)        16416     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 32, 64, 32)       0         
 2D)                                                             
                            

C:\Users\user\AppData\Roaming\Python\Python310\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


45/45 [==============================] - ETA: 0s - loss: 0.2718
Epoch 1: val_loss improved from inf to 0.16927, saving model to best_weights.h5
45/45 [==============================] - 21s 449ms/step - loss: 0.2718 - val_loss: 0.1693
Epoch 2/10
45/45 [==============================] - ETA: 0s - loss: 0.0742
Epoch 2: val_loss improved from 0.16927 to 0.03835, saving model to best_weights.h5
45/45 [==============================] - 24s 528ms/step - loss: 0.0742 - val_loss: 0.0383
Epoch 3/10
45/45 [==============================] - ETA: 0s - loss: 0.0263
Epoch 3: val_loss improved from 0.03835 to 0.01909, saving model to best_weights.h5
45/45 [==============================] - 24s 525ms/step - loss: 0.0263 - val_loss: 0.0191
Epoch 4/10
45/45 [==============================] - ETA: 0s - loss: 0.0143
Epoch 4: val_loss improved from 0.01909 to 0.01155, saving model to best_weights.h5
45/45 [==============================] - 22s 500ms/step - loss: 0.0143 - val_loss: 0.0116
Epoch 5/10
45/45 [=

C:\Users\user\AppData\Roaming\Python\Python310\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


45/45 [==============================] - ETA: 0s - loss: 0.3224
Epoch 1: val_loss improved from inf to 0.23865, saving model to best_weights.h5
45/45 [==============================] - 24s 513ms/step - loss: 0.3224 - val_loss: 0.2387
Epoch 2/10
45/45 [==============================] - ETA: 0s - loss: 0.1078
Epoch 2: val_loss improved from 0.23865 to 0.04795, saving model to best_weights.h5
45/45 [==============================] - 23s 518ms/step - loss: 0.1078 - val_loss: 0.0480
Epoch 3/10
45/45 [==============================] - ETA: 0s - loss: 0.0288
Epoch 3: val_loss improved from 0.04795 to 0.01914, saving model to best_weights.h5
45/45 [==============================] - 23s 511ms/step - loss: 0.0288 - val_loss: 0.0191
Epoch 4/10
45/45 [==============================] - ETA: 0s - loss: 0.0142
Epoch 4: val_loss improved from 0.01914 to 0.01169, saving model to best_weights.h5
45/45 [==============================] - 24s 530ms/step - loss: 0.0142 - val_loss: 0.0117
Epoch 5/10
45/45 [=

...

C:\Users\user\AppData\Roaming\Python\Python310\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


34/34 [==============================] - ETA: 0s - loss: 0.2747
Epoch 1: val_loss improved from inf to 0.22381, saving model to best_weights.h5
34/34 [==============================] - 18s 516ms/step - loss: 0.2747 - val_loss: 0.2238
Epoch 2/10
34/34 [==============================] - ETA: 0s - loss: 0.1325
Epoch 2: val_loss improved from 0.22381 to 0.06674, saving model to best_weights.h5
34/34 [==============================] - 16s 482ms/step - loss: 0.1325 - val_loss: 0.0667
Epoch 3/10
34/34 [==============================] - ETA: 0s - loss: 0.0517
Epoch 3: val_loss improved from 0.06674 to 0.03852, saving model to best_weights.h5
34/34 [==============================] - 16s 474ms/step - loss: 0.0517 - val_loss: 0.0385
Epoch 4/10
34/34 [==============================] - ETA: 0s - loss: 0.0286
Epoch 4: val_loss improved from 0.03852 to 0.02217, saving model to best_weights.h5
34/34 [==============================] - 17s 495ms/step - loss: 0.0286 - val_loss: 0.0222
Epoch 5/10
34/34 [=

C:\Users\user\AppData\Roaming\Python\Python310\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


34/34 [==============================] - ETA: 0s - loss: 0.2816
Epoch 1: val_loss improved from inf to 0.19988, saving model to best_weights.h5
34/34 [==============================] - 17s 486ms/step - loss: 0.2816 - val_loss: 0.1999
Epoch 2/10
34/34 [==============================] - ETA: 0s - loss: 0.1075
Epoch 2: val_loss improved from 0.19988 to 0.04800, saving model to best_weights.h5
34/34 [==============================] - 18s 528ms/step - loss: 0.1075 - val_loss: 0.0480
Epoch 3/10
34/34 [==============================] - ETA: 0s - loss: 0.0375
Epoch 3: val_loss improved from 0.04800 to 0.02842, saving model to best_weights.h5
34/34 [==============================] - 17s 486ms/step - loss: 0.0375 - val_loss: 0.0284
Epoch 4/10
34/34 [==============================] - ETA: 0s - loss: 0.0201
Epoch 4: val_loss improved from 0.02842 to 0.01478, saving model to best_weights.h5
34/34 [==============================] - 18s 529ms/step - loss: 0.0201 - val_loss: 0.0148
Epoch 5/10
34/34 [=

KeyboardInterrupt: 

# Prodicting PDO

In [6]:
#import the average trend of global SST 
gsst = xr.open_dataarray('C:/Users/user/Research/Research Code/CNN/Data/'+'ersst_1854_2022_mech.nc')
gclm = gsst.groupby('time.month').mean(dim='time')
ganm = (gsst.groupby('time.month') - gclm)
mglobal=ganm.mean(("lon", "lat"), skipna=True)
plt.plot(mglobal)
plt.show

<function matplotlib.pyplot.show(*, block=None)>

In [7]:
##Setup validation period
ystr = 1983
yend = 2022

##Setup lat dimension
lead_time = 359
N_train = 1428

##Imported ensemble training results used to transform land mask
ncin = Dataset(ifile+'predictions_tuned_leadtime'+str(lead_time)+'params_'+str(N_train)+'_10_'+str(lead_time)+'.nc')


sst  = ncin.variables['__xarray_dataarray_variable__'][:,:-9,:] 
time  = ncin.variables['time'][:]
lat  = ncin.variables['lat'][:-9]
lon  = ncin.variables['lon'][:]
lev  = ncin.variables['lev'][:]
ncin.close()

nt,nlat,nlon,nlev = sst.shape    
print(sst.shape )

## Imported pred data
ds = xr.open_dataset(ifile+'predictions_tuned_leadtime'+str(lead_time)+'params_'+str(N_train)+'_10_'+str(lead_time)+'.nc')
ds = ds.sel(lat=slice(70, 20), lon=slice(122, 250))
print(ds)

(480, 23, 64, 1)
<xarray.Dataset>
Dimensions:                        (time: 480, lat: 23, lon: 64, lev: 1)
Coordinates:
  * time                           (time) datetime64[ns] 1983-01-01 ... 2022-...
  * lat                            (lat) float32 64.0 62.0 60.0 ... 22.0 20.0
  * lon                            (lon) float32 122.0 124.0 ... 246.0 248.0
  * lev                            (lev) float64 0.0
Data variables:
    __xarray_dataarray_variable__  (time, lat, lon, lev) float32 ...


In [8]:
##import land mask
lmfile = 'data\lsmask.nc'
lmset  = Dataset(lmfile)
lsmask = lmset['mask'][0,24:70:2,122:250:2]# read land mask
lsmask = lsmask-1

num_repeats = nt
lsm = np.stack([lsmask]*num_repeats,axis=-1).transpose((2,0,1))
lsm.shape

(480, 23, 64)

In [9]:
a = np.arange('1983-01', '2023-01', dtype='datetime64[M]')

In [10]:
# === Climatology and Anomalies 
sst = ds.__xarray_dataarray_variable__
clm = sst.sel(time=slice(f'{ystr}-01-01',f'{yend}-12-01')).groupby('time.month').mean(dim='time')
anm = (sst.groupby('time.month') - clm)

# remove lev dimension
anm=anm.isel(lev=0)

##Prediction
#detrend
dpdo = detrend_dim(anm,mglobal,'time',1)

#Mask out land
sst_diff = dpdo
sst_diff = np.ma.masked_array(sst_diff, mask=lsm)
sst_diff[lsm<0] = np.nan
#sst_diff=sst_diff.to_numpy()
print(sst_diff.shape)

(480, 23, 64)


In [11]:
#To perform the EOF analysis, create an EOF solver.
#Prior to computing the EOFs, latitude weights are applied with a cosine.
wgts   = np.sqrt(np.cos(np.deg2rad(lat)))
wgts   = wgts.reshape(len(wgts), 1)
wgts.shape

#Obtained the leading EOFs
solver = Eof(sst_diff, weights=wgts)

eof1 = solver.eofs(neofs=1)
pc1  = solver.pcs(npcs=1, pcscaling=1)
varfrac = solver.varianceFraction()
lambdas = solver.eigenvalues()
Covariance_eof1 = solver.eofsAsCovariance(neofs=1, pcscaling=0)

leading_eof = solver.eofsAsCorrelation(neofs=1)
eigenvalue1 = solver.eigenvalues(neigs=1)

In [12]:
#Creat PDO index
pc=pc1 [:,]
pc_mean=pc1[:,].mean()
pc_std=pc1[:,].std()

#Normalised
pcs = xr.apply_ufunc(

lambda x, m, s: x  / s,
pc,
pc_mean,
pc_std, dask = 'allowed', vectorize = True)
pcs.shape
PDO=pcs[:,0]

(480, 1)

In [14]:
#Smoothed with 10-year low pass
cutoff = 0.05
lw_pred = lowpass_filter(cutoff ,PDO)

np.savetxt(outdir_smooth_pred+'Result_PDO index_pred_cwcnn_smoothed'+str(lead_time)+'.csv',lw_pred, delimiter=',', fmt='%s')


In [15]:
##Plot EOF map and PC Amplitude
parallels = np.arange(-90,90,30.)
meridians = np.arange(-180,180,30)

for i in range(0,1):
    fig = plt.figure(figsize=(3,3))
    plt.subplot(211)
    
    m = Basemap(projection='cyl', llcrnrlon=min(lon), llcrnrlat=min(lat), urcrnrlon=max(lon), urcrnrlat=max(lat))   
    x, y = m(*np.meshgrid(lon, lat))
    clevs = np.linspace(np.min(eof1[i,:,:].squeeze()), np.max(eof1[i,:,:].squeeze()), 21)
    cs = m.contourf(x, y, eof1[i,:,:].squeeze(), clevs, cmap=plt.cm.RdBu_r)
    m.drawcoastlines()  
    m.drawparallels(parallels, labels=[1,0,0,0])
    m.drawmeridians(meridians, labels=[1,0,0,1])

    cb = m.colorbar(cs, 'right', size='5%', pad='2%')
    cb.set_label('EOF', fontsize=12)
    plt.title('EOF ' + str(i+1), fontsize=16)

    plt.subplot(212)
    days = np.linspace(1983,2022,nt)
    plt.plot(days, pc1[:,i], linewidth=2)
    plt.axhline(0, color='k')
    plt.xlabel('Year')
    plt.ylabel('PC Amplitude')   
    plt.ylim(np.min(pc1.squeeze()), np.max(pc1.squeeze()))

In [7]:
##Calculate correlation

##Import validation data from NCEI PDO index from 1983-2022 
#https://www.ncei.noaa.gov/access/monitoring/pdo/
cutoff = 0.05
PDO_ncei = pd.read_excel(ifile_data+'NCEI PDO index _overall.xlsx',header=None,index_col=None,
                      skiprows=985, skipfooter = 0,
                      usecols=[1]  
                     )

PDO_ncei = adjust(PDO_ncei)
lw_ncei = lowpass_filter(cutoff ,PDO_ncei)
PDO_ncei=adjust(lw_ncei)


##Calculate the average correlation over 30 years for lead forecasts.
lead_Correlation_cnn = []

for lead_time in range (0,360):
    
    Correlation_cnn = []
    
    PDO_cnn = np.loadtxt(outdir_smooth_pred+'Result_PDO index_pred_cwcnn_smoothed'+str(lead_time)+'.csv', dtype=float)
    PDO_pred = adjust(PDO_cnn)

    
    for i in range (0,480): 
        correlation(PDO_pred,PDO_ncei,Correlation_cnn)

    lead_Correlation_cnn.append(mean(Correlation_cnn[0:481]))

print(mean(lead_Correlation_cnn))

C:\Users\User\anaconda3\lib\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
C:\Users\User\anaconda3\lib\site-packages\numpy\core\_methods.py:182: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
C:\Users\User\anaconda3\lib\site-packages\numpy\lib\function_base.py:2845: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\User\anaconda3\lib\site-packages\numpy\lib\function_base.py:2704: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
C:\Users\User\anaconda3\lib\site-packages\numpy\lib\function_base.py:2704: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


0.82


# END